## Decompress the zip package and preprocess the csv file.

In [16]:
Dataset_path_CICIDS_2017 = 'data_raw/CICIDS2017'

import os
file_zip = os.path.join(Dataset_path_CICIDS_2017, 'GeneratedLabelledFlows.zip')
assert os.path.exists(file_zip)


In [17]:

import zipfile
file_out = os.path.join(Dataset_path_CICIDS_2017, 'GeneratedLabelledFlows')
if not os.path.exists(file_out):
    with zipfile.ZipFile(file_zip) as zf:
        zf.extractall(file_out)

assert os.path.exists(os.path.join(Dataset_path_CICIDS_2017,'GeneratedLabelledFlows/TrafficLabelling /Monday-WorkingHours.pcap_ISCX.csv'))

In [18]:
import os
import pandas as pd

csv_path = os.path.join(Dataset_path_CICIDS_2017,
                        'GeneratedLabelledFlows/TrafficLabelling /')

CICIDS2017_csv_name = [
    'Monday-WorkingHours.pcap_ISCX.csv',
    'Tuesday-WorkingHours.pcap_ISCX.csv',
    'Wednesday-workingHours.pcap_ISCX.csv',
    'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
    'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
    'Friday-WorkingHours-Morning.pcap_ISCX.csv',
    'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv',
    'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv'
]

CICIDS2017_csv = [pd.read_csv(os.path.join(csv_path, csv_name), encoding='cp1252') for csv_name in CICIDS2017_csv_name]

X_all = pd.concat(CICIDS2017_csv)
X_all.shape

/home/jinke_cheng/anaconda3/envs/PerFedAvg/lib/python3.7/site-packages/ipykernel_launcher.py:18: DtypeWarning: Columns (0,1,3,6,84) have mixed types.Specify dtype option on import or set low_memory=False.


(3119345, 85)

In [19]:
X_all = X_all.reset_index(drop=True)
X_all = X_all.drop(labels=[' Fwd Header Length'], axis=1)

X_all.columns = ['Flow ID', 'Src IP', 'Src Port', 'Dst IP', 'Dst Port', 'Protocol', 'Timestamp', 'Flow Duration', 'Total Fwd Packet', 'Total Bwd packets', 'Total Length of Fwd Packet', 'Total Length of Bwd Packet', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Packet Length Min', 'Packet Length Max', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg', 'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg', 'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes', 'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min', 'Activity']

y = X_all.pop('Activity')
X_all.shape, y.shape

((3119345, 83), (3119345,))

In [20]:

replacement_mapping_dict = {'BENIGN': 0,
                            'FTP-Patator': 1,
                            'SSH-Patator': 2,
                            'DoS Hulk': 3,
                            'DoS GoldenEye': 4,
                            'DoS slowloris': 5,
                            'DoS Slowhttptest': 6,
                            'Heartbleed': 7,
                            'Web Attack – Brute Force': 8,
                            'Web Attack – XSS': 9,
                            'Web Attack – Sql Injection': 10,
                            'Infiltration': 11,
                            'Bot': 12,
                            'DDoS': 13,
                            'PortScan': 14,
                            }
y = y.replace(replacement_mapping_dict, regex=True)
pd.Series(y).value_counts()


0.0     2273097
3.0      231073
14.0     158930
13.0     128027
4.0       10293
1.0        7938
2.0        5897
5.0        5796
6.0        5499
12.0       1966
8.0        1507
9.0         652
11.0         36
10.0         21
7.0          11
Name: Activity, dtype: int64

In [21]:
XY_ALL = pd.concat([X_all, y], axis=1)
XY_ALL = XY_ALL.drop(labels=['Flow ID', 'Src IP', 'Src Port',
                           'Dst IP', 'Dst Port', 'Timestamp'], axis=1)

XY_ALL = XY_ALL.drop(labels=['Fwd PSH Flags',
                           'Fwd URG Flags',
                           'Bwd URG Flags',
                           'URG Flag Count',
                           'Fwd Bytes/Bulk Avg',
                           'Fwd Packet/Bulk Avg',
                           'Fwd Bulk Rate Avg',
                           'Bwd Bytes/Bulk Avg',
                           'Bwd Packet/Bulk Avg',
                           'Bwd Bulk Rate Avg',
                           'FWD Init Win Bytes',
                           'Fwd Seg Size Min',
                           ],
                   axis=1)
XY_ALL.shape

(3119345, 66)

In [22]:
XY_ALL.shape


(3119345, 66)

In [23]:
import numpy as np

where_nan = np.where(np.isnan(XY_ALL))[0]
where_inf = np.where(XY_ALL.values >= np.finfo(np.float64).max)[0]
where_bad = np.unique(np.concatenate((where_nan, where_inf)))
XY_all = XY_ALL.drop(index = where_bad)

XY_all.to_csv(os.path.join(Dataset_path_CICIDS_2017, "CICIDS2017_pre_all_noInf.csv"), index=None)
XY_all.shape

(2827876, 66)

## Data splitting

In [24]:
import os
Dataset_path_CICIDS_2017 = 'data_raw/CICIDS2017'
args_dataset_root = "data_div/CICIDS2017"

if not os.path.exists(args_dataset_root):
    os.makedirs(args_dataset_root)
    os.makedirs(os.path.join(args_dataset_root, 'train'))
    os.makedirs(os.path.join(args_dataset_root, 'test'))


In [25]:
import pandas as pd

XY_all = pd.read_csv(os.path.join(Dataset_path_CICIDS_2017, "CICIDS2017_pre_all_noInf.csv"))

In [26]:
known_lable = [1, 2, 3, 4, 5, 6, 8, 13, 14]
train_lable = [0] + known_lable
test_lable = [9, 10, 11, 12]

XY_012345681314 = XY_all.loc[XY_all['Activity'].isin(train_lable)]
XY_9101112 = XY_all.loc[XY_all['Activity'].isin(test_lable)]

In [27]:
from sklearn.model_selection import train_test_split


def train_test_split_pop_and_concat(XY, Y=None, pop_c=None, train_size=0.8, random_state=0, return4=True):
    if pop_c:
        Y = XY.pop(pop_c)
    X_train, X_test, Y_train, Y_test = train_test_split(XY, Y, train_size=train_size, stratify=Y, random_state=random_state)  # shuffle=False
    if return4:
        return X_train, X_test, Y_train, Y_test
    else:
        XY_train = pd.concat([X_train, Y_train], axis=1)
        XY_test = pd.concat([X_test, Y_test], axis=1)
        return XY_train, XY_test


In [28]:
XY_012345681314_train, XY_012345681314_test = train_test_split_pop_and_concat(XY_012345681314, pop_c='Activity', train_size=0.66, return4=False)


In [29]:
XY_012345681314_train.shape, XY_012345681314_test.shape # 2528845	6	5063650

((1864632, 66), (960568, 66))

In [30]:
def sample_dataframe(df_all, number):
    df_sample = df_all.sample(min(number, df_all.shape[0]))  # 不够number了就全取出来
    df_remain = df_all.drop(df_sample.index, axis=0)
    return df_sample, df_remain

## train clinet data generation

In [31]:
XY_train = XY_012345681314_train
pd.Series(XY_train['Activity']).value_counts()

0.0     1499071
3.0      151882
14.0     104811
13.0      84497
4.0        6793
1.0        5237
2.0        3892
5.0        3825
6.0        3629
8.0         995
Name: Activity, dtype: int64

In [32]:
import random
for clint_num in range(400):
    clint_content = pd.DataFrame()

    indexsample = random.sample(train_lable, 5)
    for index in indexsample:
        index_content = XY_train.loc[XY_train['Activity'] == index]
        index_content_sample = index_content.sample(15, replace=True)  # 每次运行sample的是不一样的
        clint_content = clint_content.append(index_content_sample)

    path = os.path.join(args_dataset_root, "train", str(clint_num) + ".csv")
    clint_content.to_csv(path)

## test client data generation

In [33]:
XY_test = pd.concat([XY_012345681314_test, XY_9101112], axis=0)
pd.Series(XY_test['Activity']).value_counts()

0.0     772249
3.0      78242
14.0     53993
13.0     43528
4.0       3500
1.0       2698
2.0       2005
5.0       1971
12.0      1956
6.0       1870
9.0        652
8.0        512
11.0        36
10.0        21
Name: Activity, dtype: int64

In [34]:

for unknown_class_number in range(5):
    known_class_number = 5 - 1 - unknown_class_number
    for clint_num in range(400+unknown_class_number*10, 400+unknown_class_number*10+10):
        sample_lable = [0] + random.sample(known_lable, known_class_number) + random.sample(test_lable, unknown_class_number)
        # print(clint_num, known_class_number, unknown_class_number, sample_lable)
            
        clint_content = pd.DataFrame()
        for index in sample_lable:
            index_content = XY_test.loc[XY_test['Activity'] == index]
            index_content_sample, _ = sample_dataframe(index_content, 100)
            clint_content = clint_content.append(index_content_sample)

        path = os.path.join(args_dataset_root, "test", str(clint_num) + ".csv")
        clint_content.to_csv(path)